<div style="font-size: 200%; font-weight: bold; color: maroon;">Clasificación de texto</div>
<!-- Paulo Villegas, 2020 - v. 1.3 -->

Montamos un clasificador binario para distinguir el idioma de tweets (inglés o español)

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

plt.rcParams['figure.figsize'] = (15, 5)

# 1. Carga de datos

In [ ]:
# Carga los datos. 
# No hace falta inferir el esquema ya que son dos columnas string
data_all = spark.read.format("csv") \
    .option("header","true").option("inferSchema","false").option("multiLine", "true") \
    .load("../data/data-tweet.csv.bz2")

In [ ]:
data_all.printSchema()

## 1.1 Inspección

In [ ]:
# ¿Cuantos tenemos?
num = data_all.count()
num

In [ ]:
# Get a sample of around 20 rows
smpl = data_all.sample(False,20.0/num).toPandas()

with pd.option_context('display.max_colwidth', 512):
    display(smpl)

In [ ]:
# ¿Cuántos por idioma?
num_bylang = data_all.groupBy("lang").count().toPandas()
num_bylang

## 1.2 Balanceado

Las clases están muy desbalanceadas. Para evitarlo vamos a remuestrear usando los ratios entre las dos clases

In [ ]:
num_bylang['count'].min() / num_bylang['count']

In [ ]:
sample_rates = dict(zip( num_bylang['lang'], num_bylang['count'].min() / num_bylang['count'] ))
sample_rates

In [ ]:
# Remuestreo
data = data_all.sampleBy('lang', sample_rates)

In [ ]:
data.groupBy("lang").count().toPandas()

# 2. Preproceso y aprendizaje

## 2.1 Preproceso de texto

Aplicamos una cadena estándar de transformación para convertir los tweets (texto) en vectores clasificables: [tokenizado](https://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html#pyspark.ml.feature.Tokenizer) + [TF](https://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html#pyspark.ml.feature.HashingTF)/[IDF](https://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html#pyspark.ml.feature.IDF)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
data_w = tokenizer.transform(data)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures",numFeatures=1<<20)
data_f = hashingTF.transform(data_w)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=100)
idfModel = idf.fit(data_f)
data_r = idfModel.transform(data_f)

In [ ]:
hashingTF.getNumFeatures()

In [ ]:
data_r.printSchema()

In [ ]:
smpl = data_r.select("lang", "words","features").limit(5).toPandas()

with pd.option_context('display.max_colwidth', 512):
    display(smpl)

## 2.2 Etiquetado

Añadimos la columna con la anotación de clase. Spark-ML exige que la anotación sea numérica: `0.0` y `1.0` para las dos clases

Usamos:

|idioma|anot |
|------|-----|
|es    |0.0  |
|en    |1.0  |

In [ ]:
from pyspark.sql.functions import udf, expr
from pyspark.sql.types import IntegerType, DoubleType

udf_mapper = udf( lambda x: 1.0 if x == 'en' else 0.0, returnType=DoubleType() )

data_l = data_r.withColumn( 'label', udf_mapper(data.lang) )

In [ ]:
data_l.printSchema()

In [ ]:
data_l.limit(10).toPandas()

## 2.3 Aprendizaje

In [ ]:
# Creamos el modelo
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

model = LogisticRegression(maxIter=50, regParam=0.001, featuresCol="features" )
#model = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
#model = RandomForestClassifier(numTrees=40)

In [ ]:
# Entrenamos el modelo
%time model_f = model.fit(data_l)

## 2.4 Resultados

In [ ]:
# Calculamos la predicción para nuestro dataset
data_p = model_f.transform( data_l )

In [ ]:
# Obtenemos la matriz de confusión
cf = data_p.groupBy('label','prediction').count()
cf.collect()

In [ ]:
# Lo mismo, pero mejor presentado (vía Pandas)
cf.toPandas().pivot(index='label',columns='prediction',values='count')

In [ ]:
data_p.dtypes

In [ ]:
# Veamos los errores
columns = ['lang', 'label', 'prediction', 'text', 'probability']
err = data_p.filter( (data_p['label'] == 1.0) & (data_p['prediction'] == 0.0) ).toPandas()

with pd.option_context('display.max_colwidth', 512):
    display(err[columns])

# 3. Grabación de modelos

## 3.1 por separado

Grabamos los modelos a fichero para luego poder cargarlos en el script que los aplica en _streaming_

Sólo necesitamos grabar los bloques **que han sido entrenados** : `IDFModel` y `LogisticRegression`. Los otros dos (`Tokenizer`y `HashingTF`) no necesitan ajustarse a los datos.

In [ ]:
# Graba los modelos que han sido entrenados
idfModel.write().overwrite().save("models/tweet-classifier-idf")
model_f.write().overwrite().save("models/tweet-classifier-lr")

## 3.2 en pipeline

Una variante interesante es crear un `Pipeline` con todo el proceso y grabarlo, en vez de bloque a bloque. Esto permite cargarlo luego de una vez.

In [ ]:
from pyspark.ml.pipeline import Pipeline

# Creamos un objeto Pipeline
pl = Pipeline()

# Añadimos todos 5 pasos de nuestra cadena de proceso.
pl.setStages( [tokenizer, hashingTF, idfModel, model_f] )

# Obtenemos el pipeline entrenado
plFitted = pl.fit( data )

# Grabamos a disco
plFitted.write().overwrite().save( "models/tweet-classifier")

### 3.2.1 Carga del pipeline

In [ ]:
# Cargarlo luego es muy sencillo
from pyspark.ml.pipeline import PipelineModel
pl2 = PipelineModel.load("models/tweet-classifier")

In [ ]:
data_res = pl2.transform( data )

In [ ]:
data_res.limit(3).toPandas()